## Combine Output Files if Necessary - Skip if not used

In [1]:
import pandas as pd
import numpy as np
import time
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import warnings
warnings.filterwarnings('ignore')

t0=time.time()

firstRd=pd.read_csv("../csvResults/cityPatviewLatLongAddrLatLongAgtLatLong.csv",sep="\t")
secondRd=pd.read_csv("../csvResults/cityPatviewLatLongAddrLatLongAgtLatLongDataLatLong_50000_65752.csv")
df=pd.concat([firstRd,secondRd],axis=0).reset_index(drop=True)

t1=time.time()
total=t1-t0
print("Total time is %4f" % (total/60), "mins")

display(df.info(null_counts=True),df.head())

Total time is 0.006917 mins
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66384 entries, 0 to 66383
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      66384 non-null  int64  
 1   assignee_id             66384 non-null  object 
 2   location_id             66384 non-null  object 
 3   city                    65449 non-null  object 
 4   state                   65867 non-null  object 
 5   latitude                65994 non-null  float64
 6   longitude               65994 non-null  float64
 7   organization            66384 non-null  object 
 8   dateOfFirstPat          66384 non-null  object 
 9   name                    48982 non-null  object 
 10  jurisdiction_code       48982 non-null  object 
 11  incorporation_date      48982 non-null  object 
 12  alternative_names       5343 non-null   object 
 13  previous_names          7302 non-null   object 
 14  address_ci

None

,ID,assignee_id,location_id,city,state,latitude,longitude,organization,dateOfFirstPat,name,...,address_city_latitude,address_city_longitude,agent_city,agent_state,agent_city_latitude,agent_city_longitude,data_city,data_state,data_city_latitude,data_city_longitude
0,0,fffe9f1f-cb1c-49ab-b00f-6416e3e3a909,fd8b2b76-cb90-11eb-9615-121df0c29c1e,austin,tx,30.3076,-97.7126,"close-in solutions, llc",10/18/2011,"close in solutions, llc",...,30.5229,-97.6776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,"valley business solutions, llc",6/30/2020,valley payroll management and business solutio...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,"valley business solutions, llc",6/30/2020,valley business solutions llc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,fffd9c21-3bb1-4471-b316-d172921e3f83,ec16f9be-cb90-11eb-9615-121df0c29c1e,san diego,ca,32.8247,-117.1520,"railias holdings, llc",12/15/2020,"railias holdings, llc",...,NaN,NaN,san diego,ca,32.8247,-117.152,NaN,NaN,NaN,NaN
4,6,fffa8520-f0c2-431f-a64a-e16f3af0a896,fc25b086-cb8f-11eb-9615-121df0c29c1e,cincinnati,oh,39.1366,-84.5135,fairdale orthodontic company,9/22/1987,fairdale orthodontic company,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Extract Alternative and Previous Names

In [2]:
# import pandas as pd
# import numpy as np
# import time
# import re
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process
# import warnings
# warnings.filterwarnings('ignore')

# t0=time.time()

# df=pd.read_csv("../csvResults/cityPatviewLatLongAddrLatLongAgtLatLong_50000_65752.csv",sep="\t")

# t1=time.time()
# total=t1-t0
# print("Total time is %4f" % (total/60), "mins")

# display(df.info(null_counts=True),df.head())

In [3]:
### consecutive regex processes were performed to extract the organization name from the
### alternative_names and previous_names columns. When a match is absent, the if statement
### skips that record. If a match is found, the first regex variable is used to find all
### instances that match the pattern, then the second regex variable is utilized to further
### refine the string to reduce it to the organization name. A list is created using the
### nested for loop for each record. If multiple names are present, a nested list is created
### for that record. After the for loops are finished running, the final list is created and
### printed for reviewing. The description applies for processing both the alternative_names
### and previous_names columns.

regex="'company_name': '[a-z].+?'"
regex1="'[a-z].+': "

b=len(df)
sub_finalAlt = []
finalAlt = []

for j in range(b):
    
    if pd.isna(df['alternative_names'][j]) is True:
        pass
    
    else:
        a=df['alternative_names'][j][2:-2]
        match = re.findall(regex, a)
        c=len(match)

        for i in range(c):
            match1 = re.sub(regex1,"",match[i])
            match1=match1.replace("'","").title()
            sub_finalAlt.append(match1)

    finalAlt.append(sub_finalAlt)
    sub_finalAlt = []

print(finalAlt)

[['Close In Solutions, Llc'], [], [], [], [], [], ['Maxmile Technologies, Llc'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['Fina Technology, Inc.', 'Cosden Technology, Inc.', 'Fina Technology, Inc.'], [], [], [], ['Advanced Temperature Devices, Inc.'], ['Tencate Grass North America', 'Greenfields Usa'], [], [], [], ['Orteq Energy Services, Llc', 'Orteq Energy Technologies, Llc'], [], [], [], [], [], [], [], [], [], [], [], [], ['Abb Randall Corporation', 'C-E/Randall, Inc.', 'The Randall Corporation'], [], ['Jasminesola Co.'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['Bit Lion Llc'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['Hamilton Sewing Supply'], [], [], [], [], [], ['

In [4]:
b=len(df)
sub_finalPre = []
finalPre = []

for j in range(b):
    
    if pd.isna(df['previous_names'][j]) is True:
        pass
        
    else:
        a=df['previous_names'][j][2:-2]
        match = re.findall(regex, a)
        c=len(match)

        for i in range(c):
            match1 = re.sub(regex1,"",match[i])
            match1=match1.replace("'","").title()
            sub_finalPre.append(match1)

    finalPre.append(sub_finalPre)
    sub_finalPre = []

print(finalPre)

[[], [], [], [], [], [], [], [], [], [], ['Arjang & Co Inc.'], [], [], ['Electro-Materials Corp. Of America'], [], ['Mcneil Laboratories, Incorporated'], [], [], [], [], ['Energy Efficient Wall Systems, L.L.C.', 'Benchmark Energy Efficient Wall Systems, L.L.C.'], [], ['Institute For Self Defense, Inc.'], [], [], [], [], [], [], [], [], ['Leonardo Optical Corp.'], [], [], [], [], [], [], [], ['Modern Mold And Tool Corp.'], [], [], [], [], [], [], [], [], [], [], ['Bluemont Software, Inc.'], [], [], ['Lerner New York Outlet, Inc.', 'Jasmine Company, Inc.'], ['Lerner New York, Inc.', 'Lerner Stores, Inc.', 'Milton Acquisition Corp.'], [], [], [], [], [], [], [], [], [], [], ['C B D & K, Inc.'], [], [], ['Dong Ri U.S.A. Inc.'], [], ['Hua Ri Shun Trading Inc.'], ['Man Ri Jang Sung Restaurant Inc.'], ['Doo Ri Inc.'], ['Ohkwa-Ri Associates, Inc.'], ['Mi-Ri Contractors, Inc.'], ['Ri Drains Inc.'], ['Lain Ri Inc.'], ['Ri Dry Cleaners Inc.'], ['Jamph-Ri Inc.'], ['De - Ri - Su, Inc.'], ['New Woo 

In [5]:
df['alternative_names_clean'] = finalAlt
df['previous_names_clean'] = finalPre

display(df.info(),df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66384 entries, 0 to 66383
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       66384 non-null  int64  
 1   assignee_id              66384 non-null  object 
 2   location_id              66384 non-null  object 
 3   city                     65449 non-null  object 
 4   state                    65867 non-null  object 
 5   latitude                 65994 non-null  float64
 6   longitude                65994 non-null  float64
 7   organization             66384 non-null  object 
 8   dateOfFirstPat           66384 non-null  object 
 9   name                     48982 non-null  object 
 10  jurisdiction_code        48982 non-null  object 
 11  incorporation_date       48982 non-null  object 
 12  alternative_names        5343 non-null   object 
 13  previous_names           7302 non-null   object 
 14  address_city          

None

,ID,assignee_id,location_id,city,state,latitude,longitude,organization,dateOfFirstPat,name,...,agent_city,agent_state,agent_city_latitude,agent_city_longitude,data_city,data_state,data_city_latitude,data_city_longitude,alternative_names_clean,previous_names_clean
0,0,fffe9f1f-cb1c-49ab-b00f-6416e3e3a909,fd8b2b76-cb90-11eb-9615-121df0c29c1e,austin,tx,30.3076,-97.7126,"close-in solutions, llc",10/18/2011,"close in solutions, llc",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Close In Solutions, Llc]",[]
1,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,"valley business solutions, llc",6/30/2020,valley payroll management and business solutio...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
2,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,"valley business solutions, llc",6/30/2020,valley business solutions llc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
3,4,fffd9c21-3bb1-4471-b316-d172921e3f83,ec16f9be-cb90-11eb-9615-121df0c29c1e,san diego,ca,32.8247,-117.1520,"railias holdings, llc",12/15/2020,"railias holdings, llc",...,san diego,ca,32.8247,-117.152,NaN,NaN,NaN,NaN,[],[]
4,6,fffa8520-f0c2-431f-a64a-e16f3af0a896,fc25b086-cb8f-11eb-9615-121df0c29c1e,cincinnati,oh,39.1366,-84.5135,fairdale orthodontic company,9/22/1987,fairdale orthodontic company,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]


In [6]:
### drop the alternative_names and previous_names columns and rearrange the data so that
### the alternative_names_clean and previous_names_clean features are next to the name
### feature

df1=df
df1.drop(labels=['alternative_names','previous_names'],axis=1,inplace=True)

ten_col = df1.pop('alternative_names_clean')
ele_col = df1.pop('previous_names_clean')

df1.insert(10, 'alternative_names_clean', ten_col)
df1.insert(11, 'previous_names_clean', ele_col)

display(df1.info(),df1.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66384 entries, 0 to 66383
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       66384 non-null  int64  
 1   assignee_id              66384 non-null  object 
 2   location_id              66384 non-null  object 
 3   city                     65449 non-null  object 
 4   state                    65867 non-null  object 
 5   latitude                 65994 non-null  float64
 6   longitude                65994 non-null  float64
 7   organization             66384 non-null  object 
 8   dateOfFirstPat           66384 non-null  object 
 9   name                     48982 non-null  object 
 10  alternative_names_clean  66384 non-null  object 
 11  previous_names_clean     66384 non-null  object 
 12  jurisdiction_code        48982 non-null  object 
 13  incorporation_date       48982 non-null  object 
 14  address_city          

None

,ID,assignee_id,location_id,city,state,latitude,longitude,organization,dateOfFirstPat,name,...,address_city_latitude,address_city_longitude,agent_city,agent_state,agent_city_latitude,agent_city_longitude,data_city,data_state,data_city_latitude,data_city_longitude
0,0,fffe9f1f-cb1c-49ab-b00f-6416e3e3a909,fd8b2b76-cb90-11eb-9615-121df0c29c1e,austin,tx,30.3076,-97.7126,"close-in solutions, llc",10/18/2011,"close in solutions, llc",...,30.5229,-97.6776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,"valley business solutions, llc",6/30/2020,valley payroll management and business solutio...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,"valley business solutions, llc",6/30/2020,valley business solutions llc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,fffd9c21-3bb1-4471-b316-d172921e3f83,ec16f9be-cb90-11eb-9615-121df0c29c1e,san diego,ca,32.8247,-117.1520,"railias holdings, llc",12/15/2020,"railias holdings, llc",...,NaN,NaN,san diego,ca,32.8247,-117.152,NaN,NaN,NaN,NaN
4,6,fffa8520-f0c2-431f-a64a-e16f3af0a896,fc25b086-cb8f-11eb-9615-121df0c29c1e,cincinnati,oh,39.1366,-84.5135,fairdale orthodontic company,9/22/1987,fairdale orthodontic company,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Clean the Names in the Organization and Name Fields

In [7]:
#capitalize first letter in all words and strip whitespace around individual strings
df2=df1
df2['organization']=df2['organization'].str.title().str.lstrip().str.rstrip()
df2['name']=df2['name'].str.title().str.rstrip().str.rstrip()

#remove suffixes from organization names
df2['organization'].replace({" Corp\.$| Corporation":"",", Inc\.$|, Inc$| Inc\.$| Inc$| Incorporation":"",
                             ", Incorporated|Incorporated":""," Co\.$|Co\.$":" company",
                             ", L\.L\.C|, L\.L\.C\.|, L\.l\.c\.| L\.L\.C\.| Llc|, Llc|, Llc\.":"",
                             ", Ltd\.| Ltd\.,| Ltd\.| Ltd":"", " L\.P\.|, L\.P\.|, Lp| Lp":"",
                             ", Plc| Plc":"",", Pllc| Pllc":"",", L\.L\.L\.P\.":""," Lc|, Lc":"",
                             ", P\.C\.| P\.C\.":""," & ":" and ",", Llp| Llp|, L\.L\.P\.":"",
                             "\.$":"",",":"","!":"","@":"","#":""},
                             regex=True,inplace=True)

df2['organization'].replace({" Corporation":""," Incorporation":""},regex=True,inplace=True)

df2['name'].replace({" Corp\.$| Corporation":"",", Inc\.$|, Inc$| Inc\.$| Inc$| Incorporation":"",
                     ", Incorporated|Incorporated":""," Co\.$|Co\.$":" company",
                     ", L\.L\.C|, L\.L\.C\.|, L\.l\.c\.| L\.L\.C\.| Llc|, Llc|, Llc\.":"",
                     ", Ltd\.| Ltd\.,| Ltd\.| Ltd":"", " L\.P\.|, L\.P\.|, Lp| Lp":"",
                     ", Plc| Plc":"",", Pllc| Pllc":"",", L\.L\.L\.P\.":""," Lc|, Lc":"",
                     ", P\.C\.| P\.C\.":""," & ":" and ",", Llp| Llp|, L\.L\.P\.":"",
                     "\.$":"",",":"","!":"","@":"","#":""},
                     regex=True,inplace=True)

df2['name'].replace({" Corporation":""," Incorporation":""},regex=True,inplace=True)

df2['organization']=df2['organization'].str.lower()
df2['name']=df2['name'].str.lower()

df2.head()

,ID,assignee_id,location_id,city,state,latitude,longitude,organization,dateOfFirstPat,name,...,address_city_latitude,address_city_longitude,agent_city,agent_state,agent_city_latitude,agent_city_longitude,data_city,data_state,data_city_latitude,data_city_longitude
0,0,fffe9f1f-cb1c-49ab-b00f-6416e3e3a909,fd8b2b76-cb90-11eb-9615-121df0c29c1e,austin,tx,30.3076,-97.7126,close-in solutions,10/18/2011,close in solutions,...,30.5229,-97.6776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,6/30/2020,valley payroll management and business solutions,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,6/30/2020,valley business solutions,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,fffd9c21-3bb1-4471-b316-d172921e3f83,ec16f9be-cb90-11eb-9615-121df0c29c1e,san diego,ca,32.8247,-117.1520,railias holdings,12/15/2020,railias holdings,...,NaN,NaN,san diego,ca,32.8247,-117.152,NaN,NaN,NaN,NaN
4,6,fffa8520-f0c2-431f-a64a-e16f3af0a896,fc25b086-cb8f-11eb-9615-121df0c29c1e,cincinnati,oh,39.1366,-84.5135,fairdale orthodontic company,9/22/1987,fairdale orthodontic company,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
### standardize suffixes in the alternative_names_clean and previous_names_clean list

b=len(df2)

for j in range(b):
    d=len(df2.iloc[j,10])
    
    try:
        if d == 0:
            pass

        elif d == 1:
            
            df2.iloc[j,10][0]=re.sub(r", Inc\.$| Inc\.$|, Inc$| Inc$| Incorporation$|, Incoporation$ \
                                     |, Incorporated$| Incorporated$", "", df2.iloc[j,10][0])
            df2.iloc[j,10][0]=re.sub(r", Llc$| Llc$|, Llc\.$| Llc\.$| L\.L\.C\.$|, L\.L\.C\.$", "", df2.iloc[j,10][0])
            df2.iloc[j,10][0]=re.sub(r", Co\.$| Co\.$|, Co$| Co$| Co\., The", " company", df2.iloc[j,10][0])
            df2.iloc[j,10][0]=re.sub(r"& Co|and Co|& Co\.|and Co\.", "and company", df2.iloc[j,10][0])
            df2.iloc[j,10][0]=re.sub(r" Corp\.$|, Corp\.$|, Corp$| Corp$|, Corporation| Corporation", "", df2.iloc[j,10][0])
            df2.iloc[j,10][0]=re.sub(r", L\.P\.$| L\.P\.$|, Lp$| Lp$", "", df2.iloc[j,10][0])
            df2.iloc[j,10][0]=re.sub(r", Ltd\.$| Ltd\.$|, Ltd$| Ltd$", "", df2.iloc[j,10][0])
            df2.iloc[j,10][0]=re.sub(r", L\.C\.$| L\.C\.$", "", df2.iloc[j,10][0])
            df2.iloc[j,10][0]=re.sub(r" & ", " and ", df2.iloc[j,10][0])
            df2.iloc[j,10][0]=re.sub(r", The", "", df2.iloc[j,10][0])
            df2.iloc[j,10][0]=re.sub(r" \([A-Z].*\)$", "", df2.iloc[j,10][0])
            df2.iloc[j,10][0]=re.sub(r",|!|@|#", "", df2.iloc[j,10][0])
            df2.iloc[j,10][0]=df2.iloc[j,10][0].lower()

            print(df2.iloc[j,10][0])

        elif d > 1:

            for t in range(d):
                
                df2.iloc[j,10][t]=re.sub(r", Inc\.$| Inc\.$|, Inc$| Inc$| Incorporation$|, Incoporation$ \
                                     |, Incorporated$| Incorporated$", "", df2.iloc[j,10][t])
                df2.iloc[j,10][t]=re.sub(r", Llc$| Llc$|, Llc\.$| Llc\.$| L\.L\.C\.$|, L\.L\.C\.$", "", df2.iloc[j,10][t])
                df2.iloc[j,10][t]=re.sub(r", Co\.$| Co\.$|, Co$| Co$| Co\., The", " company", df2.iloc[j,10][t])
                df2.iloc[j,10][t]=re.sub(r"& Co|and Co|& Co\.|and Co\.", "and company", df2.iloc[j,10][t])
                df2.iloc[j,10][t]=re.sub(r" Corp\.$|, Corp\.$|, Corp$| Corp$|, Corporation| Corporation", "", df2.iloc[j,10][t])
                df2.iloc[j,10][t]=re.sub(r", L\.P\.$| L\.P\.$|, Lp$| Lp$", "", df2.iloc[j,10][t])
                df2.iloc[j,10][t]=re.sub(r", Ltd\.$| Ltd\.$|, Ltd$| Ltd$", "", df2.iloc[j,10][t])
                df2.iloc[j,10][t]=re.sub(r", L\.C\.$| L\.C\.$", "", df2.iloc[j,10][t])
                df2.iloc[j,10][t]=re.sub(r" & ", " and ", df2.iloc[j,10][t])
                df2.iloc[j,10][t]=re.sub(r", The", "", df2.iloc[j,10][t])
                df2.iloc[j,10][t]=re.sub(r" \([A-Z].*\)$", "", df2.iloc[j,10][t])
                df2.iloc[j,10][t]=re.sub(r",|!|@|#", "", df2.iloc[j,10][t])
                df2.iloc[j,10][t]=df2.iloc[j,10][t].lower()

                print(df2.iloc[j,10][t])

    except:
        print('Something is very wrong')

close in solutions
maxmile technologies
fina technology
cosden technology
fina technology
advanced temperature devices
tencate grass north america
greenfields usa
orteq energy services
orteq energy technologies
abb randall
c-e/randall
the randall
jasminesola company
bit lion
hamilton sewing supply
biobex
biobex
compaq computer
gateway technology
compaq computer
gateway technology
butler taper joint
microseismic
microseismic
boulder acquisition
affiliated computer services
acs unclaimed property clearinghouse
affiliated computer services
xerox business services
affiliated computer services
conduent business services
xerox business services
conduent business services
acs business resources
business resources
acs bpo services
conduent bpo services
conduent bpo services
acs enterprise solutions
r e search
business records dba greet america public records services
acs government information services
acs state and local services
acs government records
acs technical services
business records


national educational assessments
communication specialists
csi publishing
unique publishing
digital production services
capital spectrum
print isle
digital spectrum services
communication specialists
texas mailing and printing company
printing craft
csi
zack print
csi digiprint
csi mailplus
national educational assessments
capital spectrum
communication specialists
csi publishing
unique publishing
digital production services
nissi international
spectrum digital
capital spectrum
digital imaging
unique publishing
photo graphics
digital production services
austin cs-1985
aa capital
round rock lime company
the westminister
the westminister
round rock lime company
round rock white lime company
walsh enterprises
round rock white lime company
agc automotive americas company
agc automotive company
agc automotive replacement glass
agc automotive replacement glass
agc business development americas
agc glass company north america
agc glass company north america
ap technoglass company
maret pharma

south atlantic insurance services of tx
cowan-hill bond agency
insurance and bonds agency of texas
ibtx risk services
smith manus
smith-manus
smith manus surety bonds
indemco
m.w. insurance services
ward and moore insurance services
ward and moore insurance services
the signature b&b companies
beckerman and companympany
rrl insurance agency
fletcher and companympany
carlisle insurance agency
carlisle insurance
international irm
bria insurance and risk consultants
chelf insurance-mckay agency
mckay insurance group
mckay insurance
mckay insurance
williams insurance and financial services
williams business services
industrial insurance associates
r.n. jones insurance agency
american casualty insurance agency
ralph nannola insurance agency
shubert and associates insurance
britton gallagher and associates
falcon insurance agency
falcon insurance agency of houston
menger underwriting services
halton hall and associates an insurance agency
halton hall and associates
new home insurance
new hom

sentara family medicine physicians
sentara surgery specialists
sentara pulmonary and critical care specialists
sentara physical medicine and pain management specialists
sentara vascular specialists
sentara podiatry specialists
sentara orthopaedic trauma specialists
sentara surgery specialists
sentara hearing and balance center
sentara neurosurgery specialists
sentara pediatric physicians
sentara neurology specialists
sentara surgery specialists
sentara family medicine physicians
sentara vascular specialists
sentara hand specialists
dedicated care center
sentara metabolic and weight loss surgery center
sentara dermatology specialists
sentara advanced heart failure clinic
sentara internal medicine physicians
sentara hospital medicine physicians
sentara orthopaedic trauma specialists
sentara family medicine physicians
sentara radiology specialists
sentara vascular specialists
sentara rheumatology specialists
executive evaluation center
sentara heart arrhythmia center
sentara infectious di

suspended vortex innovations
leisure lectronics
azota gas processing
rippling
fcstone environmental
frank j. schuh
escalade sports
starfish retention solutions inc.
sorenson forensics
genetree
flo trend systems
cytex plastics
gearench
yesop
opo company
gearench manufacturing company
gearench sales
orbix i
orbix i
orbix
gearench
petol-gearench
petol gearench
protell systems international
psi technologies systems
affinity cellular
ready mobile
wavcatcher
weatherford holding u.s.
enterra oil field rental
oil field rental service company
enterra oil field rental company
weatherford enterra u.s.
weatherford u.s.
oil field rental manufacturing services
enterra oil field rental manufacturing services
central machine
m. w. pollock investments
pollock advisory group
financial gravity wealth
financial gravity wealth
sofos investments
financial gravity asset management
pollock investments
poltex
pollock corrugated containers
texas paper company
gulf envelope company
poltex
gulf envelope company
p

...

tele-source
proform fitness products
icon fit elements
weslo
legend sporting goods
healthrider
workout warehouse
fitness warehouse
dow jones and companympany
dow jones and companympany
dow jones and companympany
qugence
omnitracs
omnitracs
sellers manufacturing
standard renewable energy
standard renewable energy
netpliance
shbang
sea arama marineworld
sea sell
sea gull motel
village by the sea
village by the sea
village by the sea apartments
village by the sea
village by the sea apartments
sea ranch
sea meadows
allied chemical
texas natural storage company
union texas
java sea
north sea/texas 1982
sea sands
sea barge
sea fishers-1979
sea properties
sea properties
north sea/texas 1982
sea arama marineworld
sea barge
village by the sea
village by the sea
village by the sea apartments
village by the sea
village by the sea apartments
allied chemical
texas natural storage company
union texas
sea meadows
sea sands
sea ranch
java sea
sea gull motel
sea fishers-1979
sea sell
sea/tac properties

diversified home health services
diversified home medical equipment
downriver cancer center
downriver center for oncology
downriver surgeons
electrophysiology consultants
electrophysiology consultants
eyes on art
eyes on classic design
eyes on design
eyes on design/detroit
eyes on the classics
eyeson design
eyeson design at the north american international auto show
fairlane home infusion therapy
fairlane home infusion therapy
fairlane home medical supply
fitnessworks
fort knox market square
friends of vision
golf fling
greenfield health systems
greenfield health systems
hamtramck health center
hamtramck health center
harris stroke center
healthcore
healthcore
henry ford ambulatory pharmacy services
henry ford at home
henry ford athletic medicine at cottage
henry ford behavioral health
henry ford behavioral health
henry ford behavioral health - cottage
henry ford behavioral health services - cottage hospital
henry ford behavioral health services maplegrove center
henry ford behavioral 

international abm
live oak intellectual property
bison
systemix research
westlake intellectual property
westlake technology
westlake intellectual property
westlake intelectual property
westlake intellectual property
westlake c and e
westlake intelectual property
active expert
texas network academy
l3 expert
wifi training
aln online
active life network
stay active network
john svoboda
bobinvention
post n mail
e-stamp
e-stamp
fed devices
nextio
c/m international
cmi - southfield
cmi - southfield
cmi - southfield
cmi - southfield
c/m international
ibn technology holding
tyco electronics installation services
lineage power installation services
tyco electronics power systems
lineage power
ge power electronics
abb power electronics
tyco electronics power systems
lineage power
ge power electronics
abb power electronics
tyco electronics installation services
lineage power installation services
rock solid images
harmonious designs
harmonious designs
rev clean power
rev clean power
rev clean po

tri-state environmental
huron fluid power
huron mechanical contracting
shore-tec
shores hydraulics
shores technologies
south bank marine
south bank marine
st. clair development company
st. clair development company
chamber notes
fastprint
great northern distribution
harbor beach times
home seekers
independent press
jeffersonian
north branch gazette
official michigan
port austin times
sanilac county bargain hunter
sanilac county jeffersonian
sanilac county news
sanilac mini storage
sanilac publishing
sanilac publishing
sanilac web printing
sanilac web printing
st. clair shopper
the brown city banner
the county line reminder
the county press
the huron county press
the newsweekly
the north branch news
the sand pointer
the senior voice
the thumb of the mitten home book
the thumb of the mitten home buyers guide
detroit - metro harley - davidson company
mariners river lodge
mariners river lodge
st. clair river lodge
extrusion technologies
huron extruded products
huron products
huron products

weatherford pawn
texas tin shop
tin
tin star investments
tin star investment
tin snips
tin cup management
tin barn
net-tin
tin hoi restaurant
bar-tin enterprises
tin man
becker oil tools
denbury onshore
derby industries
itec steel
nusteel commercial
nucon steel commercial
kenxus investments
kenxus
ifly cinematography
ifly cinematography
ndustri.com
i3 plastic cards
ufollowit
eistream acquisition
eistream
eistream
eistream viewstar
eistream wms
greenlight
greenlight
greenlight
swipejobs
swipejobs
swipejobs
swipejobs
i3 quality metric
element w
uplogix
iaeration
symplr


In [9]:
### standardize suffixes in the alternative_names_clean and previous_names_clean list
b=len(df2)

for j in range(b):
    d=len(df2.iloc[j,11])
    
    try:
        if d == 0:
            pass

        elif d == 1:

            df2.iloc[j,11][0]=re.sub(r", Inc\.$| Inc\.$|, Inc$| Inc$| Incorporation$|, Incoporation$ \
                                     |, Incorporated$| Incorporated$", "", df2.iloc[j,11][0])
            df2.iloc[j,11][0]=re.sub(r", Llc$| Llc$|, Llc\.$| Llc\.$| L\.L\.C\.$|, L\.L\.C\.$", "", df2.iloc[j,11][0])
            df2.iloc[j,11][0]=re.sub(r", Co\.$| Co\.$|, Co$| Co$| Co\., The", " company", df2.iloc[j,11][0])
            df2.iloc[j,11][0]=re.sub(r"& Co|and Co|& Co\.|and Co\.", "and company", df2.iloc[j,11][0])
            df2.iloc[j,11][0]=re.sub(r" Corp\.$|, Corp\.$|, Corp$| Corp$|, Corporation| Corporation", "", df2.iloc[j,11][0])
            df2.iloc[j,11][0]=re.sub(r", L\.P\.$| L\.P\.$|, Lp$| Lp$", "", df2.iloc[j,11][0])
            df2.iloc[j,11][0]=re.sub(r", Ltd\.$| Ltd\.$|, Ltd$| Ltd$", "", df2.iloc[j,11][0])
            df2.iloc[j,11][0]=re.sub(r", L\.C\.$| L\.C\.$", "", df2.iloc[j,11][0])
            df2.iloc[j,11][0]=re.sub(r" & ", " and ", df2.iloc[j,11][0])
            df2.iloc[j,11][0]=re.sub(r", The", "", df2.iloc[j,11][0])
            df2.iloc[j,11][0]=re.sub(r" \([A-Z].*\)$", "", df2.iloc[j,11][0])
            df2.iloc[j,11][0]=re.sub(r",|!|@|#", "", df2.iloc[j,11][0])
            df2.iloc[j,11][0]=df2.iloc[j,11][0].lower()

            print(df2.iloc[j,11][0])


        elif d > 1:

            for t in range(d):

                df2.iloc[j,11][t]=re.sub(r", Inc\.$| Inc\.$|, Inc$| Inc$| Incorporation$|, Incoporation$ \
                                     |, Incorporated$| Incorporated$", "", df2.iloc[j,11][t])
                df2.iloc[j,11][t]=re.sub(r", Llc$| Llc$|, Llc\.$| Llc\.$| L\.L\.C\.$|, L\.L\.C\.$", "", df2.iloc[j,11][t])
                df2.iloc[j,11][t]=re.sub(r", Co\.$| Co\.$|, Co$| Co$| Co\., The", " company", df2.iloc[j,11][t])
                df2.iloc[j,11][t]=re.sub(r"& Co|and Co|& Co\.|and Co\.", "and company", df2.iloc[j,11][t])
                df2.iloc[j,11][t]=re.sub(r" Corp\.$|, Corp\.$|, Corp$| Corp$|, Corporation| Corporation", "", df2.iloc[j,11][t])
                df2.iloc[j,11][t]=re.sub(r", L\.P\.$| L\.P\.$|, Lp$| Lp$", "", df2.iloc[j,11][t])
                df2.iloc[j,11][t]=re.sub(r", Ltd\.$| Ltd\.$|, Ltd$| Ltd$", "", df2.iloc[j,11][t])
                df2.iloc[j,11][t]=re.sub(r", L\.C\.$| L\.C\.$", "", df2.iloc[j,11][t])
                df2.iloc[j,11][t]=re.sub(r" & ", " and ", df2.iloc[j,11][t])
                df2.iloc[j,11][t]=re.sub(r", The", "", df2.iloc[j,11][t])
                df2.iloc[j,11][t]=re.sub(r" \([A-Z].*\)$", "", df2.iloc[j,11][t])
                df2.iloc[j,11][t]=re.sub(r",|!|@|#", "", df2.iloc[j,11][t])
                df2.iloc[j,11][t]=df2.iloc[j,11][t].lower()

                print(df2.iloc[j,11][0])

    except:
        df2.iloc[j,11][t]='Something is very wrong'


arjang and company
electro-materials corp. of america
mcneil laboratories
energy efficient wall systems
energy efficient wall systems
institute for self defense
leonardo optical
modern mold and tool
bluemont software
lerner new york outlet
lerner new york outlet
lerner new york
lerner new york
lerner new york
c b d and k
dong ri u.s.a.
hua ri shun trading
man ri jang sung restaurant
doo ri
ohkwa-ri associates
mi-ri contractors
ri drains
lain ri
ri dry cleaners
jamph-ri
de - ri - su
new woo ri house
chong ryang ri house
oberon ri
jo-ri deli
kilt of ri
kilt of ri
lain ri development
ri hillcroft
ri and di
ri beverages
dan-ri universal
dan-ri universal
pc tech u.s.a. and you-ri
pc tech u.s.a. and you-ri
lo-ri-an holding
jin ri trading
li ri construction
brenner mcdonagh tortolani
ri communications
ri du
chris-ri construction company
fortune five packaging
poseidon aqua products
cold spring harbor laboratory association
cold spring harbor laboratory association
former brick company
former 

carrier sand and gravel
depandable carrier
fleet carrier
dmi liquidation
dmi liquidation
dmi liquidation
fuel carrier
the arrow carrier
hempstead carrier
hempstead carrier
carrier international
carrier international
image carrier
carrier bus
carrier bus
carrier case of america
purolator courier
purolator courier
purolator courier
carrier foundation
carrier foundation
carrier foundation
neptune supermart
neptune supermart
carrier communications
carrier capital
carlyle credit
carlyle credit
carlyle credit
gilbert carrier
american carrier-call
folding carrier
firestone diversified products
joliejewels
spheros north america
open water power
open water power
txteagle
natural organics services
natural organics advertising
natural body organics
natural organics laboratories
natural organics laboratories
natural organics
natural organics store.com
libbey-owens glass company
libbey-owens glass company
libbey-owens glass company
libbey-owens glass company
libbey-owens glass company
libbey-owens 

abex
abex
abex
american brake shoe company
american brake shoe company
american brake shoe company
msw
msw
causley-mcinerney
causley-mcinerney
mcinerney-miller brothers
dough babies
pacific scientific instruments company
pacific scientific instruments company
deaconess professional practic
harvard medical school-beth is
harvard medical school-beth is
beth israel hospital obstetric
beth israel hospital association
nitrocell biosciences
discovery pharmaceuticals
doyle vacuum cleaner company
emu solutions
hge enterprises
element k online holdings
element k online holdings
element k online holdings
element k online
element k online
element k online
biogen
biogen
biogen
biogen
inteva products sterbfritz
pitman contracting co. of kans.
pitman contracting co. of kans.
pitman contracting co. of kans.
pitman contracting co. of kans.
pitman contracting co. of kans.
pitman contracting co. of kans.
pitman contracting co. of kans.
pitman contracting co. of kans.
pitman contracting co. of kans.
jimi

stoller architecture p.c.
d. krag
accuvisiondesigns
mightytechs
ceo enterprises
howmedica international
howmedica international
howmedica international
ritsema-millgard
luck quarries
punkinfutz
philco engineering
ahura
ahura
synchronicity inc. of massach
critical care concepts
critical concepts
ferrell-ross
ferrell-ross
interactive data pricing and reference data
interactive data pricing and reference data
neurophage pharmaceuticals
content savvy
oraline
immtek
immtek
nec systems laboratory
altec
breakaway imaging
indian industries
ricar industries
neural impact
yaskin starfish enterprises
turf usa
integri-test
copeland companyrporation
adl management education insti
adl management education insti
flx
flx
flx
yonder music
dayton t. brown
dayton t. brown
total air-conditioning
total air
arley
arley
houghton mifflin company
okay acquisition
siemens medical solutions usa
milbank tweed hadley and mccloy llp
mustad
mustad
express magazine
express magazine
vsi international
carolina manufact

dirigo
dirigo
international technology
open box technologies
air reduction company
specialty paperboard
earth satellite earthsat
earth satellite earthsat
ambulatory monitoring
ambulatory monitoring services
alu-tech systems
healthsource insurance company
parker isaac instruments
system analysis
accessories
accessories
accessories
ethyl overseas development
ethyl overseas development
ethyl overseas development
ethyl development
ethyl development
ethyl development
ethyl development
ethyl development
vca
vca
louis berkman winter products company
the louis berkman snow and ice company
federal-mogul-bower bearings
federal-mogul-bower bearings
bigwood technology
ocrolus i
ocrolus i
ocrolus i
hitchcock chair company the
fabreeka products company
fabreeka products company
hillyard chemical company
hillyard chemical company
thermorefinery technologies
big brothers big sisters of the ocean state
big brothers big sisters of the ocean state
big brothers big sisters of the ocean state
exeter volunt

...

mortronics
mortronics
mortronics
national feedscrew and machining industries
aqualizer
cabot foundation
cabot engineering company
quilting it
quilting it
caeden
apollo genetics
blether labs
ned strongin associates
ned strongin associates
jaffe acoustics
jaffe acoustics
jaffe acoustics
jaffe acoustics
jaffe acoustics
paul c. rodusky
yellow freight system inc. of delaware
yellow freight system inc. of delaware
yellow freight system inc. of delaware
yellow freight system inc. of delaware
yellow freight system inc. of delaware
yellow technology services
yellow technology services
yellow technology services
yellow technology services
yellow technology services
machine vision international
machine vision international
machine vision international
data genomix
data genomix
hickok operating
sickles golf
air-cure environmental
air-cure environmental
gfi group
gfi capital resources group
gfi capital resources group
qwasi
qwasi media
ce-pe-seeinc.
ce-pe-seeinc.
sas see maid
say see bone music
say

arcus data security
iron mountain information serv
iron mountain information serv
iron mountain information serv
dsi technology escrow services
iron mountain information part
mountain iron and supply
bhrm acquisition
iron mountain information serv
iron mountain information serv
iron mountain information serv
iron mountain information serv
iron mountain information serv
dsi technology escrow services
iron mountain information part
bhrm acquisition
arcus data security
mountain iron and supply
arcus data security
dsi technology escrow services
iron mountain information serv
iron mountain information part
bhrm acquisition
iron mountain information serv
iron mountain information serv
iron mountain information serv
mountain iron and supply
parks-cramer company
parks-cramer company
parks-cramer company
parks-cramer company
reshydro
general heat and power
general heat and power
general heat and power
general heat and power
meyer enterprises ii
meyer-columbia enterprises
m. jonas meyer enterpri

the clark-elliot company
hubbard-elliott copper company
elliott hardwood company
elliott hardwood company
elliott company
the chas. h. elliott company
the clark-elliot company
dale elliott and company
dale elliott and company
dale elliott and company
the cleveland-jeffries automobile company
the cleveland-jeffries automobile company
hubbard-elliott copper company
the chas. h. elliott company
elliott hardwood company
elliott hardwood company
e and e home appliance company
e and e home appliance company
bryan-elliott company
debower-elliott company
elliott ticket company
elliott company
mckimmy and elliot distributing company
mckimmy and elliot distributing company
mckimmy and elliot distributing company
the gilbert elliott law company
elliott gill and companympany
b.k. elliott company
duesler and elliott trucking company
dymo business systems
dymo business systems
dymo business systems
dymo business systems
dymo business systems
elliott enterprises
elliott enterprises
tabor agency
tabor

at-a-glance
at-a-glance
at-a-glance
tissue engineering
frankenmuth-forrest twp. archers
frankenmuth-forrest twp. archers
glotech international
versa tech engineering company
lincoln laser company
kitefarms
kitefarms
alka farms
chiu technical
chiu technical
dynamic machine works
etco building systems
racemark international
racemark international
racemark international
racemark international
racemark international
racemark international
foundation for behavioral research
marquette general hospital
oakland hospital
hoover container sales and services
pro-pharmaceuticals
temperpak technologies
i-scan
contech construction products
contech construction products
pioneer precision products company
msi products
compuware florida development
compuware hockey club
compuware hockey club
computer hockey
compuware hockey club
compuware hockey club
compuware florida development
computer hockey
indemnis systems
indemnis systems
ihc hospitals
health services of the church of jesus christ of
health serv

w. t. sherrill
innovid
pullen/neier
negro league legends
broadway legends
two legends restaurant
legends bar
post house legends
international legends
international legends
reflections of legends
legends new york
legends and heroes collectibles
huntington square legends
sports legends sports
a and j legends
legends processing
legends sportswear
rock legends
authentic legends
authentic legends
cowboy legends
legends comics and cards
legends are forever
legends
m.s. legends
unlocked legends
unlocked legends
hoop legends
legends video
legends
legends
auburn legends
legends grill
legends beekman
legends sports cafe
legends tavern
storybox legends
ames true temper
drackett chemical sales company
dmt
max factor company
rll
rll
domo associates
home data source
home data source
drone
drone
drone
drone
grote and hartmann america
grote and hartmann america
goodmarc designs
neighborcom
hipcricket
hipcricket
dslt
dslt
dslt
better your credit company
the better model form company
better lawn care co

## Score the Organization Names for Patentsview against the OC Results

In [10]:
### score the organization names against the name, alternative_names_clean, and previous_names_clean features

t0=time.time()

df2['organization']=df2['organization'].str.lstrip()
df2['organization']=df2['organization'].str.rstrip()

a=len(df)
mat1=[]
mat2=[]

for i in range(a):
    
    ### try/except is used to bypass cells with an NaN. Removing this will force the user
    ### to deal with exception errors
    try:
        ### calculate the fuzzy score for further evaluation
        q=fuzz.ratio(df2['organization'][i], df2['name'][i])

        ### most simplest and most representative in the data; if the score is 100, it is
        ### a perfect match, otherwise, the remaining code will resolve score differences
        if q == 100:
            mat1.append(q)
            mat2.append(df2['name'][i])

        elif ( q != 100 ):
            ### calculate the size of the data in each row for the alternative_names_clean
            ### column and previous_names_clean column
            r=len(df2['alternative_names_clean'][i])
            d=len(df2['previous_names_clean'][i])

            ### the remaining parts of the code uses if statements to step through the many
            ### conditions that may be present in the data. As each conditions is satisfied,
            ### the mat1 and mat2 lists are appended with the data; take note that q is the
            ### original score and s, v, e, and f are separate scores that are compared
            ### against q
            if r == 0 and d == 0:
                mat1.append(q)
                mat2.append(df2['name'][i])
            
            ### calculate the score for the alternative_names_clean and previous_names_clean
            ### columns. Once calculated, they are compared against each other and q to
            ### determine the score that is highest. The 'best' score is appened to mat1 and
            ### the name of the organization is appended to mat2. The same scorer is utilized 
            ### as the above but the process.extractOne function retrieves the organization
            ### match with the highest score and saves it as a tuple
            elif r == 1 and d == 1:
                s=process.extractOne(df2['organization'][i], df2['alternative_names_clean'][i],scorer=fuzz.ratio)
                e=process.extractOne(df2['organization'][i], df2['previous_names_clean'][i],scorer=fuzz.ratio)
                
                if s[1] > e[1] and s[1] > q:
                    mat1.append(s[1])
                    mat2.append(s[0])
                
                elif e[1] > s[1] and e[1] > q:
                    mat1.append(e[1])
                    mat2.append(e[0])
                
                elif e[1] == q or s[1] == q:
                    mat1.append(q)
                    mat2.append(df2['name'][i])
                
                else:
                    mat1.append(q)
                    mat2.append(df2['name'][i])
            
            ### this section is the same as above, except the code is looking at the
            ### alternative_names_clean column only
            elif r == 1:
                s=process.extractOne(df2['organization'][i], df2['alternative_names_clean'][i],scorer=fuzz.ratio)
                
                if s[1] > q:
                    mat1.append(s[1])
                    mat2.append(s[0])
                
                elif s[1] == q:
                    mat1.append(q)
                    mat2.append(df2['name'][i])
                    
                else:
                    mat1.append(q)
                    mat2.append(df2['name'][i])
            
            ### this section is the same as above, except the code is looking at the
            ### previous_names_clean column only
            elif d == 1:
                e=process.extractOne(df2['organization'][i], df2['previous_names_clean'][i],scorer=fuzz.ratio)

                if e[1] > q:
                    mat1.append(e[1])
                    mat2.append(e[0])
                
                elif e[1] == q:
                    mat1.append(q)
                    mat2.append(df2['name'][i])
                    
                else:
                    mat1.append(q)
                    mat2.append(df2['name'][i])      
            
            ### the following two sections resolve r and d lengths greater than 1 (i.e.,
            ### records that have more than 1 company names in the alternative_names_clean 
            ### and previous_names_clean columns)
            elif r > 1:
                v=process.extractOne(df2['organization'][i], df2['alternative_names_clean'][i],scorer=fuzz.ratio)                
            
                if v[1] > q:
                    mat1.append(v[1])
                    mat2.append(v[0])
                    
                elif v[1] == q:
                    mat1.append(q)
                    mat2.append(df2['name'][i])
                    
                else:
                    mat1.append(q)
                    mat2.append(df2['name'][i])
            
            elif d > 1:
                f=process.extractOne(df2['organization'][i], df2['previous_names_clean'][i],scorer=fuzz.ratio)
                
                if f[1] > q:
                    mat1.append(f[1])
                    mat2.append(f[0])
                    
                elif f[1] == q:
                    mat1.append(q)
                    mat2.append(df2['name'][i])                    
                    
                else:
                    mat1.append(q)
                    mat2.append(df2['name'][i])

    except:
        ### error handling that places an NaN for every cells that does not have a value
        ### in the mat1 and/or mat2 lists
        mat1.append(np.nan)
        mat2.append(np.nan)

In [11]:
### concatenates the mat1 and mat2 lists into a dataframe

finalList=[list(w) for w in zip(mat1, mat2)]
finalListDf=pd.DataFrame(finalList,columns=['scores','names'])

display(finalListDf.info(),finalListDf.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66384 entries, 0 to 66383
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   scores  48982 non-null  float64
 1   names   48982 non-null  object 
dtypes: float64(1), object(1)
memory usage: 1.0+ MB


None

,scores,names
0,100.0,close in solutions
1,68.0,valley payroll management and business solutions
2,100.0,valley business solutions
3,100.0,railias holdings
4,100.0,fairdale orthodontic company


In [12]:
### appends the dataframe in which the organization names came from with the
### results from the scoring steps in the preceeding code blocks

df2['nameScores'] = finalListDf['scores']
df2['matchNames'] = finalListDf['names']

display(df2.info(),df2.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66384 entries, 0 to 66383
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       66384 non-null  int64  
 1   assignee_id              66384 non-null  object 
 2   location_id              66384 non-null  object 
 3   city                     65449 non-null  object 
 4   state                    65867 non-null  object 
 5   latitude                 65994 non-null  float64
 6   longitude                65994 non-null  float64
 7   organization             66384 non-null  object 
 8   dateOfFirstPat           66384 non-null  object 
 9   name                     48982 non-null  object 
 10  alternative_names_clean  66384 non-null  object 
 11  previous_names_clean     66384 non-null  object 
 12  jurisdiction_code        48982 non-null  object 
 13  incorporation_date       48982 non-null  object 
 14  address_city          

None

,ID,assignee_id,location_id,city,state,latitude,longitude,organization,dateOfFirstPat,name,...,agent_city,agent_state,agent_city_latitude,agent_city_longitude,data_city,data_state,data_city_latitude,data_city_longitude,nameScores,matchNames
0,0,fffe9f1f-cb1c-49ab-b00f-6416e3e3a909,fd8b2b76-cb90-11eb-9615-121df0c29c1e,austin,tx,30.3076,-97.7126,close-in solutions,10/18/2011,close in solutions,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,close in solutions
1,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,6/30/2020,valley payroll management and business solutions,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.0,valley payroll management and business solutions
2,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,6/30/2020,valley business solutions,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,valley business solutions
3,4,fffd9c21-3bb1-4471-b316-d172921e3f83,ec16f9be-cb90-11eb-9615-121df0c29c1e,san diego,ca,32.8247,-117.1520,railias holdings,12/15/2020,railias holdings,...,san diego,ca,32.8247,-117.152,NaN,NaN,NaN,NaN,100.0,railias holdings
4,6,fffa8520-f0c2-431f-a64a-e16f3af0a896,fc25b086-cb8f-11eb-9615-121df0c29c1e,cincinnati,oh,39.1366,-84.5135,fairdale orthodontic company,9/22/1987,fairdale orthodontic company,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,fairdale orthodontic company


In [13]:
### moved the scores and match names

df3=df2

ten_col1 = df3.pop('nameScores')
ele_col1 = df3.pop('matchNames')

df3.insert(12, 'nameScores', ten_col1)
df3.insert(13, 'matchNames', ele_col1)

display(df3.info(),df3.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66384 entries, 0 to 66383
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       66384 non-null  int64  
 1   assignee_id              66384 non-null  object 
 2   location_id              66384 non-null  object 
 3   city                     65449 non-null  object 
 4   state                    65867 non-null  object 
 5   latitude                 65994 non-null  float64
 6   longitude                65994 non-null  float64
 7   organization             66384 non-null  object 
 8   dateOfFirstPat           66384 non-null  object 
 9   name                     48982 non-null  object 
 10  alternative_names_clean  66384 non-null  object 
 11  previous_names_clean     66384 non-null  object 
 12  nameScores               48982 non-null  float64
 13  matchNames               48982 non-null  object 
 14  jurisdiction_code     

None

,ID,assignee_id,location_id,city,state,latitude,longitude,organization,dateOfFirstPat,name,...,address_city_latitude,address_city_longitude,agent_city,agent_state,agent_city_latitude,agent_city_longitude,data_city,data_state,data_city_latitude,data_city_longitude
0,0,fffe9f1f-cb1c-49ab-b00f-6416e3e3a909,fd8b2b76-cb90-11eb-9615-121df0c29c1e,austin,tx,30.3076,-97.7126,close-in solutions,10/18/2011,close in solutions,...,30.5229,-97.6776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,6/30/2020,valley payroll management and business solutions,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,6/30/2020,valley business solutions,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,fffd9c21-3bb1-4471-b316-d172921e3f83,ec16f9be-cb90-11eb-9615-121df0c29c1e,san diego,ca,32.8247,-117.1520,railias holdings,12/15/2020,railias holdings,...,NaN,NaN,san diego,ca,32.8247,-117.152,NaN,NaN,NaN,NaN
4,6,fffa8520-f0c2-431f-a64a-e16f3af0a896,fc25b086-cb8f-11eb-9615-121df0c29c1e,cincinnati,oh,39.1366,-84.5135,fairdale orthodontic company,9/22/1987,fairdale orthodontic company,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# df3.to_csv("../csvResults/patentsviewOCResultsScoredNamesCombined.csv",index=False)

In [15]:
### removed the columns below to minimize the dataframe size

df4=df3

df4.drop(labels=['name','alternative_names_clean','previous_names_clean'],axis=1,inplace=True)

display(df4.info(),df4.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66384 entries, 0 to 66383
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      66384 non-null  int64  
 1   assignee_id             66384 non-null  object 
 2   location_id             66384 non-null  object 
 3   city                    65449 non-null  object 
 4   state                   65867 non-null  object 
 5   latitude                65994 non-null  float64
 6   longitude               65994 non-null  float64
 7   organization            66384 non-null  object 
 8   dateOfFirstPat          66384 non-null  object 
 9   nameScores              48982 non-null  float64
 10  matchNames              48982 non-null  object 
 11  jurisdiction_code       48982 non-null  object 
 12  incorporation_date      48982 non-null  object 
 13  address_city            33330 non-null  object 
 14  address_state           32200 non-null

None

,ID,assignee_id,location_id,city,state,latitude,longitude,organization,dateOfFirstPat,nameScores,...,address_city_latitude,address_city_longitude,agent_city,agent_state,agent_city_latitude,agent_city_longitude,data_city,data_state,data_city_latitude,data_city_longitude
0,0,fffe9f1f-cb1c-49ab-b00f-6416e3e3a909,fd8b2b76-cb90-11eb-9615-121df0c29c1e,austin,tx,30.3076,-97.7126,close-in solutions,10/18/2011,100.0,...,30.5229,-97.6776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,6/30/2020,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,6/30/2020,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,fffd9c21-3bb1-4471-b316-d172921e3f83,ec16f9be-cb90-11eb-9615-121df0c29c1e,san diego,ca,32.8247,-117.1520,railias holdings,12/15/2020,100.0,...,NaN,NaN,san diego,ca,32.8247,-117.152,NaN,NaN,NaN,NaN
4,6,fffa8520-f0c2-431f-a64a-e16f3af0a896,fc25b086-cb8f-11eb-9615-121df0c29c1e,cincinnati,oh,39.1366,-84.5135,fairdale orthodontic company,9/22/1987,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# df4.to_csv("../csvResults/patentsviewOCResultsScoredNamesRmAltPreNameColsCombined.csv",index=False)

## Score the City and State Matches using the Partial_Ratio Scorer

In [17]:
### this explanation applies to scoring the city field to the address_city/state and the
### agent_city/state. The fields are cleaned of whitespace on the right and left of
### the strings to enhance the matching steps. For loops are used to step through all
### records line-by-line and compare the two cities or states. If a blank record is
### present in either of the cities, the list of scores is appended with a 'NaN' instead
### of a score. This allows the for loop to continue without presenting with an error.
### When the loops are finished, the list of scores are added to the end of the dataframe.

t0=time.time()

cityAddCity=[]
cityAgtCity=[]
cityDataCity=[]
df4['city']=df4['city'].str.lstrip()
df4['city']=df4['city'].str.rstrip()
df4['address_city']=df4['address_city'].str.lstrip()
df4['address_city']=df4['address_city'].str.rstrip()
df4['agent_city']=df4['agent_city'].str.lstrip()
df4['agent_city']=df4['agent_city'].str.rstrip()
df4['data_city']=df4['data_city'].str.lstrip()
df4['data_city']=df4['data_city'].str.rstrip()

for i in range(a):
    
    try:
        cityAddCity.append(fuzz.ratio(df4['city'][i], df4['address_city'][i]))
        
    except:
        cityAddCity.append(np.nan)
    
    try:
        cityAgtCity.append(fuzz.ratio(df4['city'][i], df4['agent_city'][i]))
        
    except:
        cityAgtCity.append(np.nan)
    
    try:
        cityDataCity.append(fuzz.ratio(df4['city'][i], df4['data_city'][i]))
        
    except:
        cityDataCity.append(np.nan)

In [18]:
### add the match scores to the dataframe

df4['addCityScore'] = cityAddCity
df4['agtCityScore'] = cityAgtCity
df4['dataCityScore'] = cityDataCity

display(df4.info(),df4.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66384 entries, 0 to 66383
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      66384 non-null  int64  
 1   assignee_id             66384 non-null  object 
 2   location_id             66384 non-null  object 
 3   city                    65449 non-null  object 
 4   state                   65867 non-null  object 
 5   latitude                65994 non-null  float64
 6   longitude               65994 non-null  float64
 7   organization            66384 non-null  object 
 8   dateOfFirstPat          66384 non-null  object 
 9   nameScores              48982 non-null  float64
 10  matchNames              48982 non-null  object 
 11  jurisdiction_code       48982 non-null  object 
 12  incorporation_date      48982 non-null  object 
 13  address_city            33330 non-null  object 
 14  address_state           32200 non-null

None

,ID,assignee_id,location_id,city,state,latitude,longitude,organization,dateOfFirstPat,nameScores,...,agent_state,agent_city_latitude,agent_city_longitude,data_city,data_state,data_city_latitude,data_city_longitude,addCityScore,agtCityScore,dataCityScore
0,0,fffe9f1f-cb1c-49ab-b00f-6416e3e3a909,fd8b2b76-cb90-11eb-9615-121df0c29c1e,austin,tx,30.3076,-97.7126,close-in solutions,10/18/2011,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,NaN
1,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,6/30/2020,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,6/30/2020,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,fffd9c21-3bb1-4471-b316-d172921e3f83,ec16f9be-cb90-11eb-9615-121df0c29c1e,san diego,ca,32.8247,-117.1520,railias holdings,12/15/2020,100.0,...,ca,32.8247,-117.152,NaN,NaN,NaN,NaN,100.0,100.0,NaN
4,6,fffa8520-f0c2-431f-a64a-e16f3af0a896,fc25b086-cb8f-11eb-9615-121df0c29c1e,cincinnati,oh,39.1366,-84.5135,fairdale orthodontic company,9/22/1987,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
t0=time.time()

stateAddState=[]
stateAgtState=[]
stateDataState=[]

for i in range(a):
    
    try:
        stateAddState.append(fuzz.ratio(df['state'][i], df['address_state'][i]))
        
    except:
        stateAddState.append(np.nan)
        
    try:
        stateAgtState.append(fuzz.ratio(df['state'][i], df['agent_state'][i]))
        
    except:
        stateAgtState.append(np.nan)
    
    try:
        stateDataState.append(fuzz.ratio(df['state'][i], df['data_state'][i]))
        
    except:
        stateDataState.append(np.nan)

In [20]:
df4['stateAddScore'] = stateAddState
df4['stateAgtScore'] = stateAgtState
df4['stateDataScore'] = stateDataState

display(df4.info(),df4.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66384 entries, 0 to 66383
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      66384 non-null  int64  
 1   assignee_id             66384 non-null  object 
 2   location_id             66384 non-null  object 
 3   city                    65449 non-null  object 
 4   state                   65867 non-null  object 
 5   latitude                65994 non-null  float64
 6   longitude               65994 non-null  float64
 7   organization            66384 non-null  object 
 8   dateOfFirstPat          66384 non-null  object 
 9   nameScores              48982 non-null  float64
 10  matchNames              48982 non-null  object 
 11  jurisdiction_code       48982 non-null  object 
 12  incorporation_date      48982 non-null  object 
 13  address_city            33330 non-null  object 
 14  address_state           32200 non-null

None

,ID,assignee_id,location_id,city,state,latitude,longitude,organization,dateOfFirstPat,nameScores,...,data_city,data_state,data_city_latitude,data_city_longitude,addCityScore,agtCityScore,dataCityScore,stateAddScore,stateAgtScore,stateDataScore
0,0,fffe9f1f-cb1c-49ab-b00f-6416e3e3a909,fd8b2b76-cb90-11eb-9615-121df0c29c1e,austin,tx,30.3076,-97.7126,close-in solutions,10/18/2011,100.0,...,NaN,NaN,NaN,NaN,25.0,NaN,NaN,100.0,NaN,NaN
1,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,6/30/2020,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,6/30/2020,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,fffd9c21-3bb1-4471-b316-d172921e3f83,ec16f9be-cb90-11eb-9615-121df0c29c1e,san diego,ca,32.8247,-117.1520,railias holdings,12/15/2020,100.0,...,NaN,NaN,NaN,NaN,100.0,100.0,NaN,NaN,100.0,NaN
4,6,fffa8520-f0c2-431f-a64a-e16f3af0a896,fc25b086-cb8f-11eb-9615-121df0c29c1e,cincinnati,oh,39.1366,-84.5135,fairdale orthodontic company,9/22/1987,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# df4.to_csv("../csvResults/patentsviewOCResultsScoredNamesCitiesStatesCombined.csv",index=False)

## Calculate Distance between Cities if City Names are Different

In [1]:
import pandas as pd
import numpy as np
import time
from geopy import distance
from fuzzywuzzy import fuzz
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')

t0=time.time()

df5=pd.read_csv("../csvResults/patentsviewOCResultsScoredNamesCitiesStatesCombined.csv")

t1=time.time()
total=t1-t0
print("Total time is %4f" % (total/60), "mins")

display(df5.info(null_counts=True),df5.head())

Total time is 0.005476 mins
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66384 entries, 0 to 66383
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      66384 non-null  int64  
 1   assignee_id             66384 non-null  object 
 2   location_id             66384 non-null  object 
 3   city                    65449 non-null  object 
 4   state                   65867 non-null  object 
 5   latitude                65994 non-null  float64
 6   longitude               65994 non-null  float64
 7   organization            66384 non-null  object 
 8   dateOfFirstPat          66384 non-null  object 
 9   nameScores              48982 non-null  float64
 10  matchNames              48982 non-null  object 
 11  jurisdiction_code       48982 non-null  object 
 12  incorporation_date      48982 non-null  object 
 13  address_city            33330 non-null  object 
 14  address_st

None

,ID,assignee_id,location_id,city,state,latitude,longitude,organization,dateOfFirstPat,nameScores,...,data_city,data_state,data_city_latitude,data_city_longitude,addCityScore,agtCityScore,dataCityScore,stateAddScore,stateAgtScore,stateDataScore
0,0,fffe9f1f-cb1c-49ab-b00f-6416e3e3a909,fd8b2b76-cb90-11eb-9615-121df0c29c1e,austin,tx,30.3076,-97.7126,close-in solutions,10/18/2011,100.0,...,NaN,NaN,NaN,NaN,25.0,NaN,NaN,100.0,NaN,NaN
1,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,6/30/2020,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,6/30/2020,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,fffd9c21-3bb1-4471-b316-d172921e3f83,ec16f9be-cb90-11eb-9615-121df0c29c1e,san diego,ca,32.8247,-117.1520,railias holdings,12/15/2020,100.0,...,NaN,NaN,NaN,NaN,100.0,100.0,NaN,NaN,100.0,NaN
4,6,fffa8520-f0c2-431f-a64a-e16f3af0a896,fc25b086-cb8f-11eb-9615-121df0c29c1e,cincinnati,oh,39.1366,-84.5135,fairdale orthodontic company,9/22/1987,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
cityAddrCor=[]
cityAgtCor=[]
cityDataCor=[]

for p in range(len(df5)):
    
    try:
        cityCor = (df5.iloc[p,5], df5.iloc[p,6])
        cityAddressCor = (df5.iloc[p,15], df5.iloc[p,16])

        cityAddrCor.append(distance.distance(cityCor, cityAddressCor).miles)
    
    except:
        cityAddrCor.append(np.nan)
        
for p in range(len(df5)):
    
    try:
        cityCor = (df5.iloc[p,5], df5.iloc[p,6])
        cityAgentCor = (df5.iloc[p,19], df5.iloc[p,20])

        cityAgtCor.append(distance.distance(cityCor, cityAgentCor).miles)
    
    except:
        cityAgtCor.append(np.nan)

for p in range(len(df5)):
    
    try:
        cityCor = (df5.iloc[p,5], df5.iloc[p,6])
        cityDatCor = (df5.iloc[p,23], df5.iloc[p,24])

        cityDataCor.append(distance.distance(cityCor, cityDatCor).miles)
    
    except:
        cityDataCor.append(np.nan)

In [3]:
cityAddrCor1=[round(num, 1) for num in cityAddrCor]
cityAgtCor1=[round(num1, 1) for num1 in cityAgtCor]
cityDataCor1=[round(num1, 1) for num1 in cityDataCor]

df5['cityToAddrDistance'] = cityAddrCor1
df5['cityToAgtDistance'] = cityAgtCor1
df5['cityToDataDistance'] = cityDataCor

display(df5.info(),df5.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66384 entries, 0 to 66383
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      66384 non-null  int64  
 1   assignee_id             66384 non-null  object 
 2   location_id             66384 non-null  object 
 3   city                    65449 non-null  object 
 4   state                   65867 non-null  object 
 5   latitude                65994 non-null  float64
 6   longitude               65994 non-null  float64
 7   organization            66384 non-null  object 
 8   dateOfFirstPat          66384 non-null  object 
 9   nameScores              48982 non-null  float64
 10  matchNames              48982 non-null  object 
 11  jurisdiction_code       48982 non-null  object 
 12  incorporation_date      48982 non-null  object 
 13  address_city            33330 non-null  object 
 14  address_state           32200 non-null

None

,ID,assignee_id,location_id,city,state,latitude,longitude,organization,dateOfFirstPat,nameScores,...,data_city_longitude,addCityScore,agtCityScore,dataCityScore,stateAddScore,stateAgtScore,stateDataScore,cityToAddrDistance,cityToAgtDistance,cityToDataDistance
0,0,fffe9f1f-cb1c-49ab-b00f-6416e3e3a909,fd8b2b76-cb90-11eb-9615-121df0c29c1e,austin,tx,30.3076,-97.7126,close-in solutions,10/18/2011,100.0,...,NaN,25.0,NaN,NaN,100.0,NaN,NaN,15.0,NaN,NaN
1,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,6/30/2020,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,6/30/2020,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,fffd9c21-3bb1-4471-b316-d172921e3f83,ec16f9be-cb90-11eb-9615-121df0c29c1e,san diego,ca,32.8247,-117.1520,railias holdings,12/15/2020,100.0,...,NaN,100.0,100.0,NaN,NaN,100.0,NaN,NaN,0.0,NaN
4,6,fffa8520-f0c2-431f-a64a-e16f3af0a896,fc25b086-cb8f-11eb-9615-121df0c29c1e,cincinnati,oh,39.1366,-84.5135,fairdale orthodontic company,9/22/1987,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# df5.to_csv("../csvResults/patentsviewOCResultsScoredNamesCitiesStatesCordinatesCombined.csv",index=False)

## Score State Jurisdiction Against State Listed in Patentsview

In [5]:
d=len(df5)
jurisScore=[]

for t in range(d):
    try:
        jurisScore.append(fuzz.ratio(df5['state'][t],df5['jurisdiction_code'][t].replace('us_','')))      

    except:
        jurisScore.append(np.nan)

In [6]:
df5['jurisdictionScore'] = jurisScore

display(df5.info(),df5.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66384 entries, 0 to 66383
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      66384 non-null  int64  
 1   assignee_id             66384 non-null  object 
 2   location_id             66384 non-null  object 
 3   city                    65449 non-null  object 
 4   state                   65867 non-null  object 
 5   latitude                65994 non-null  float64
 6   longitude               65994 non-null  float64
 7   organization            66384 non-null  object 
 8   dateOfFirstPat          66384 non-null  object 
 9   nameScores              48982 non-null  float64
 10  matchNames              48982 non-null  object 
 11  jurisdiction_code       48982 non-null  object 
 12  incorporation_date      48982 non-null  object 
 13  address_city            33330 non-null  object 
 14  address_state           32200 non-null

None

,ID,assignee_id,location_id,city,state,latitude,longitude,organization,dateOfFirstPat,nameScores,...,addCityScore,agtCityScore,dataCityScore,stateAddScore,stateAgtScore,stateDataScore,cityToAddrDistance,cityToAgtDistance,cityToDataDistance,jurisdictionScore
0,0,fffe9f1f-cb1c-49ab-b00f-6416e3e3a909,fd8b2b76-cb90-11eb-9615-121df0c29c1e,austin,tx,30.3076,-97.7126,close-in solutions,10/18/2011,100.0,...,25.0,NaN,NaN,100.0,NaN,NaN,15.0,NaN,NaN,100.0
1,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,6/30/2020,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0
2,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,6/30/2020,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0
3,4,fffd9c21-3bb1-4471-b316-d172921e3f83,ec16f9be-cb90-11eb-9615-121df0c29c1e,san diego,ca,32.8247,-117.1520,railias holdings,12/15/2020,100.0,...,100.0,100.0,NaN,NaN,100.0,NaN,NaN,0.0,NaN,100.0
4,6,fffa8520-f0c2-431f-a64a-e16f3af0a896,fc25b086-cb8f-11eb-9615-121df0c29c1e,cincinnati,oh,39.1366,-84.5135,fairdale orthodontic company,9/22/1987,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0


In [7]:
df5['dateOfFirstPat'] = pd.to_datetime(df5['dateOfFirstPat'])
df5['incorporation_date'] = pd.to_datetime(df5['incorporation_date'],errors='coerce')

df5['dateDiff']=abs(df5['incorporation_date'] - df5['dateOfFirstPat'])/ timedelta(days=365)
df5['dateDiff']=[round(num, 2) for num in df5['dateDiff']]

display(df5.info(),df5.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66384 entries, 0 to 66383
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ID                      66384 non-null  int64         
 1   assignee_id             66384 non-null  object        
 2   location_id             66384 non-null  object        
 3   city                    65449 non-null  object        
 4   state                   65867 non-null  object        
 5   latitude                65994 non-null  float64       
 6   longitude               65994 non-null  float64       
 7   organization            66384 non-null  object        
 8   dateOfFirstPat          66384 non-null  datetime64[ns]
 9   nameScores              48982 non-null  float64       
 10  matchNames              48982 non-null  object        
 11  jurisdiction_code       48982 non-null  object        
 12  incorporation_date      48975 non-null  dateti

None

,ID,assignee_id,location_id,city,state,latitude,longitude,organization,dateOfFirstPat,nameScores,...,agtCityScore,dataCityScore,stateAddScore,stateAgtScore,stateDataScore,cityToAddrDistance,cityToAgtDistance,cityToDataDistance,jurisdictionScore,dateDiff
0,0,fffe9f1f-cb1c-49ab-b00f-6416e3e3a909,fd8b2b76-cb90-11eb-9615-121df0c29c1e,austin,tx,30.3076,-97.7126,close-in solutions,2011-10-18,100.0,...,NaN,NaN,100.0,NaN,NaN,15.0,NaN,NaN,100.0,1.56
1,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,2020-06-30,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,12.08
2,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,2020-06-30,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,12.67
3,4,fffd9c21-3bb1-4471-b316-d172921e3f83,ec16f9be-cb90-11eb-9615-121df0c29c1e,san diego,ca,32.8247,-117.1520,railias holdings,2020-12-15,100.0,...,100.0,NaN,NaN,100.0,NaN,NaN,0.0,NaN,100.0,0.73
4,6,fffa8520-f0c2-431f-a64a-e16f3af0a896,fc25b086-cb8f-11eb-9615-121df0c29c1e,cincinnati,oh,39.1366,-84.5135,fairdale orthodontic company,1987-09-22,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,19.94


In [8]:
# df5.to_csv("../csvResults/patentsviewOCResultsScoredNamesCitiesStatesCordinatesJurisdictionDateDiffCombined.csv",index=False)

## Scoring Algorithm

In [8]:
import pandas as pd
import numpy as np
import time
from geopy import distance
from fuzzywuzzy import fuzz

import warnings
warnings.filterwarnings('ignore')

t0=time.time()

df=pd.read_csv("../csvResults/patentsviewOCResultsScoredNamesCitiesStatesCordinatesJurisdictionDateDiffCombined.csv")

t1=time.time()
total=t1-t0
print("Total time is %4f" % (total/60), "mins")

display(df.info(null_counts=True),df.head())

Total time is 0.007164 mins
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66384 entries, 0 to 66383
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      66384 non-null  int64  
 1   assignee_id             66384 non-null  object 
 2   location_id             66384 non-null  object 
 3   city                    65449 non-null  object 
 4   state                   65867 non-null  object 
 5   latitude                65994 non-null  float64
 6   longitude               65994 non-null  float64
 7   organization            66384 non-null  object 
 8   dateOfFirstPat          66384 non-null  object 
 9   nameScores              48982 non-null  float64
 10  matchNames              48982 non-null  object 
 11  jurisdiction_code       48982 non-null  object 
 12  incorporation_date      48975 non-null  object 
 13  address_city            33330 non-null  object 
 14  address_st

None

,ID,assignee_id,location_id,city,state,latitude,longitude,organization,dateOfFirstPat,nameScores,...,agtCityScore,dataCityScore,stateAddScore,stateAgtScore,stateDataScore,cityToAddrDistance,cityToAgtDistance,cityToDataDistance,jurisdictionScore,dateDiff
0,0,fffe9f1f-cb1c-49ab-b00f-6416e3e3a909,fd8b2b76-cb90-11eb-9615-121df0c29c1e,austin,tx,30.3076,-97.7126,close-in solutions,2011-10-18,100.0,...,NaN,NaN,100.0,NaN,NaN,15.0,NaN,NaN,100.0,1.56
1,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,2020-06-30,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,12.08
2,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,2020-06-30,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,12.67
3,4,fffd9c21-3bb1-4471-b316-d172921e3f83,ec16f9be-cb90-11eb-9615-121df0c29c1e,san diego,ca,32.8247,-117.1520,railias holdings,2020-12-15,100.0,...,100.0,NaN,NaN,100.0,NaN,NaN,0.0,NaN,100.0,0.73
4,6,fffa8520-f0c2-431f-a64a-e16f3af0a896,fc25b086-cb8f-11eb-9615-121df0c29c1e,cincinnati,oh,39.1366,-84.5135,fairdale orthodontic company,1987-09-22,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,19.94


In [9]:
a=df.iloc[:,:].count(axis=1)

df['recordCnt']=round(a/10)
display(df.head())

,ID,assignee_id,location_id,city,state,latitude,longitude,organization,dateOfFirstPat,nameScores,...,dataCityScore,stateAddScore,stateAgtScore,stateDataScore,cityToAddrDistance,cityToAgtDistance,cityToDataDistance,jurisdictionScore,dateDiff,recordCnt
0,0,fffe9f1f-cb1c-49ab-b00f-6416e3e3a909,fd8b2b76-cb90-11eb-9615-121df0c29c1e,austin,tx,30.3076,-97.7126,close-in solutions,2011-10-18,100.0,...,NaN,100.0,NaN,NaN,15.0,NaN,NaN,100.0,1.56,2.0
1,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,2020-06-30,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,12.08,2.0
2,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,2020-06-30,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,12.67,2.0
3,4,fffd9c21-3bb1-4471-b316-d172921e3f83,ec16f9be-cb90-11eb-9615-121df0c29c1e,san diego,ca,32.8247,-117.1520,railias holdings,2020-12-15,100.0,...,NaN,NaN,100.0,NaN,NaN,0.0,NaN,100.0,0.73,2.0
4,6,fffa8520-f0c2-431f-a64a-e16f3af0a896,fc25b086-cb8f-11eb-9615-121df0c29c1e,cincinnati,oh,39.1366,-84.5135,fairdale orthodontic company,1987-09-22,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,19.94,2.0


In [10]:
totalScore=[]
d=len(df)
x=''
a=''

### score the fuzzy match percentages

for s in range(d):
    ### scores all records with a fuzzy match score with a 100%; account for organization
    ### name lengths (i.e., shorter names are scored lower than longer names)
    
    a=df['recordCnt'][s]
    
    if df['nameScores'][s] == 100:

        if len(df['organization'][s]) < 4:
            x=-3
        
        elif 4 <= len(df['organization'][s]) < 7:
            x=2
            
        elif 7 <= len(df['organization'][s]) < 10:
            x=4
            
        elif len(df['organization'][s]) >= 10:
            x=5
    
    ### visual inspection of the data indicates a discrete group between 95% and 100%,
    ### resulting in the next set. Name lengths are again accounted for and shorter
    ### names/scores are downweighted even more
    
    elif 95 <= df['nameScores'][s] < 100:
        
        if len(df['organization'][s]) < 4:
            x=-4
        
        elif 4 <= len(df['organization'][s]) < 7:
            x=1
            
        elif 7 <= len(df['organization'][s]) < 10:
            x=2
            
        elif len(df['organization'][s]) >= 10:
            x=3

    ### many of the fuzzy matches in this range are wrong, but there are a couple correct
    ### that should not be discounted. While the weights are not as high as the previous
    ### sections, correct matches will be given better scores than the next section
    
    elif 90 <= df['nameScores'][s] < 95:
        
        if len(df['organization'][s]) < 4:
            x=-5
        
        elif 4 <= len(df['organization'][s]) < 7:
            x=1
            
        elif 7 <= len(df['organization'][s]) < 10:
            x=2
            
        elif len(df['organization'][s]) >= 10:
            x=3

    ### very few are correct, but there are some misspellings that were not collected during
    ### the cleaning phase. Many of the records will be weighted down, but those that are
    ### longer in length will be given higher scores than names that are shorter
    
    elif 87 <= df['nameScores'][s] < 90:
        
        if len(df['organization'][s]) < 4:
            x=-6
        
        if 4 <= len(df['organization'][s]) < 7:
            x=0.5
            
        elif 7 <= len(df['organization'][s]) < 10:
            x=1
            
        elif len(df['organization'][s]) >= 10:
            x=1.5
        
    elif df['nameScores'][s] < 87:
        x=-7
        
    elif np.isnan(df['nameScores'][s]):
        x=-10

    ### scoring the different features that contain state information for each
    ### organization. The jurisdictionScore feature was created by extracting
    ### the state from the jurisdiction_code field. The jurisdiction_code
    ### feature is the primary metric used to match patentsview and OC records
    ### and therefore, given a larger weight. The stateAddScore is given the
    ### second highest weight because it is the primary address that is listed
    ### in an OC record. stateAgtScore is given the least amount of weight
    ### because the agent may not always be located at the registered address 
    ### for the organization.
    
    if df['jurisdictionScore'][s] == 100:
        x=x+4
    
    elif np.isnan(df['jurisdictionScore'][s]):
        x=x-4
    
    elif df['jurisdictionScore'][s] < 100:
        x=x-4
        
        
    if df['stateAddScore'][s] == 100:
        x=x+2

    elif np.isnan(df['stateAddScore'][s]):
        x=x+0

    elif df['stateAddScore'][s] < 100:
        x=x-2

        
    if df['stateAgtScore'][s] == 100:
        x=x+2

    elif np.isnan(df['stateAgtScore'][s]):
        x=x+0

    elif df['stateAgtScore'][s] < 100:
        x=x-2

            
    if df['stateDataScore'][s] == 100:
        x=x+2

    elif np.isnan(df['stateDataScore'][s]):
        x=x+0

    elif df['stateDataScore'][s] < 100:
        x=x-2
      
    ### the address_city feature is weighted more than the agent_city column for similar
    ### reasons stated in the states section above. Cities less than 4 characters long
    ### are penalized and gradually score better as the character length increase. Moreover,
    ### the score from fuzzy matching is used to create groups as shown below. Fuzzy
    ### scores below 90% are weighted negatively
    
    if df['addCityScore'][s] == 100:
        
        if len(df['address_city'][s]) < 4:
            x=x-3
        
        elif 4 <= len(df['address_city'][s]) < 7:
            x=x+1
            
        elif 7 <= len(df['address_city'][s]) < 10:
            x=x+3
            
        elif len(df['address_city'][s]) >= 10:
            x=x+5
        
        elif len(df['address_city'][s]) == 0:
            x=x+0
            
        elif np.isnan(df['address_city'][s]):
            x=x+0
    
        
    if df['agtCityScore'][s] == 100:

        if len(df['agent_city'][s]) < 4:
            x=x-3

        elif 4 <= len(df['agent_city'][s]) < 7:
            x=x+1

        elif 7 <= len(df['agent_city'][s]) < 10:
            x=x+3

        elif len(df['agent_city'][s]) >= 10:
            x=x+5

        elif len(df['agent_city'][s]) == 0:
            x=x+0

        elif np.isnan(df['address_city'][s]):
            x=x+0


    if df['dataCityScore'][s] == 100:

        if len(df['data_city'][s]) < 4:
            x=x-3

        elif 4 <= len(df['data_city'][s]) < 7:
            x=x+1

        elif 7 <= len(df['data_city'][s]) < 10:
            x=x+3

        elif len(df['data_city'][s]) >= 10:
            x=x+5

        elif len(df['data_city'][s]) == 0:
            x=x+0

        elif np.isnan(df['data_city'][s]):
            x=x+0
                
    
    if df['cityToAddrDistance'][s] == 0:
        x=x+4
    
    elif 0 < df['cityToAddrDistance'][s] < 10:
        x=x+3      
       
    elif 10 <= df['cityToAddrDistance'][s] < 50:
        x=x+2
    
    elif 50 <= df['cityToAddrDistance'][s] < 100:
        x=x+1
    
    elif 100 <= df['cityToAddrDistance'][s] < 200:
        x=x+0.50
        
#     elif np.nan(df['cityToAddrDistance'][s]):
#         x=x+0
    
    elif df['cityToAddrDistance'][s] >= 200:
        x=x-2
    
        
    if df['cityToAgtDistance'][s] == 0:
        x=x+4
    
    elif 0 < df['cityToAgtDistance'][s] < 10:
        x=x+3
        
    elif 10 <= df['cityToAgtDistance'][s] < 50:
        x=x+2
    
    elif 50 <= df['cityToAgtDistance'][s] < 100:
        x=x+1
    
    elif 100 <= df['cityToAgtDistance'][s] < 200:
        x=x+0.50
        
#     elif np.nan(df['cityToAgtDistance'][s]):
#         x=x+0
    
    elif df['cityToAgtDistance'][s] >= 200:
        x=x-2
        
        
    if df['cityToDataDistance'][s] == 0:
        x=x+4
    
    elif 0 < df['cityToDataDistance'][s] < 10:
        x=x+3
        
    elif 10 <= df['cityToDataDistance'][s] < 50:
        x=x+2
    
    elif 50 <= df['cityToDataDistance'][s] < 100:
        x=x+1
    
    elif 100 <= df['cityToDataDistance'][s] < 200:
        x=x+0.50
        
#     elif np.nan(df['cityToDataDistance'][s]):
#         x=x+0
    
    elif df['cityToDataDistance'][s] >= 200:
        x=x-2
        
        
    try:
        if df['dateDiff'][s] <= 3:
            x=x+3

        elif 3 < df['dateDiff'][s] <= 5:
            x=x+1
        
        elif 5 < df['dateDiff'][s] <= 10:
            x=x+0.5
            
        elif 10 < df['dateDiff'][s] <= 15:
            x=x+0.25
            
        elif df['dateDiff'][s] > 15:
            x=x+0

    except:
        print('cannot compare dates')
    
    totalScore.append(x+a)
    a=''

In [11]:
df['totalScore'] = totalScore

display(df.info(),df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66384 entries, 0 to 66383
Data columns (total 38 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      66384 non-null  int64  
 1   assignee_id             66384 non-null  object 
 2   location_id             66384 non-null  object 
 3   city                    65449 non-null  object 
 4   state                   65867 non-null  object 
 5   latitude                65994 non-null  float64
 6   longitude               65994 non-null  float64
 7   organization            66384 non-null  object 
 8   dateOfFirstPat          66384 non-null  object 
 9   nameScores              48982 non-null  float64
 10  matchNames              48982 non-null  object 
 11  jurisdiction_code       48982 non-null  object 
 12  incorporation_date      48975 non-null  object 
 13  address_city            33330 non-null  object 
 14  address_state           32200 non-null

None

,ID,assignee_id,location_id,city,state,latitude,longitude,organization,dateOfFirstPat,nameScores,...,stateAddScore,stateAgtScore,stateDataScore,cityToAddrDistance,cityToAgtDistance,cityToDataDistance,jurisdictionScore,dateDiff,recordCnt,totalScore
0,0,fffe9f1f-cb1c-49ab-b00f-6416e3e3a909,fd8b2b76-cb90-11eb-9615-121df0c29c1e,austin,tx,30.3076,-97.7126,close-in solutions,2011-10-18,100.0,...,100.0,NaN,NaN,15.0,NaN,NaN,100.0,1.56,2.0,18.00
1,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,2020-06-30,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,12.08,2.0,-0.75
2,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,huntsville,al,34.7015,-86.5766,valley business solutions,2020-06-30,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,12.67,2.0,11.25
3,4,fffd9c21-3bb1-4471-b316-d172921e3f83,ec16f9be-cb90-11eb-9615-121df0c29c1e,san diego,ca,32.8247,-117.1520,railias holdings,2020-12-15,100.0,...,NaN,100.0,NaN,NaN,0.0,NaN,100.0,0.73,2.0,26.00
4,6,fffa8520-f0c2-431f-a64a-e16f3af0a896,fc25b086-cb8f-11eb-9615-121df0c29c1e,cincinnati,oh,39.1366,-84.5135,fairdale orthodontic company,1987-09-22,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,19.94,2.0,11.00


In [12]:
# df.to_csv("../csvResults/screenScoredRecords.csv",index=False)

In [13]:
# df[df.duplicated(subset=['assignee_id'],keep=False)].to_csv("../csvResults/screenScoredRecordsDuplicates1.csv",index=False)